In [ ]:
import pandas as pd

# Charger les données
df = pd.read_csv("Airline_Dataset.csv")
# conversion des noms de colonnes en minuscule
df.columns = df.columns.str.lower()
df.drop(["departure delay in minutes"], axis=1, inplace=True)
df.drop(["id"], axis=1, inplace=True)
# Créer une liste des colonnes catégorielles
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/', '_')
# Filtrer les lignes avec des zéros dans les colonnes spécifiées
cols_to_filter = ['departure_arrival_time_convenient', 'ease_of_online_booking', 'gate_location', 'food_and_drink', 'online_boarding', 'seat_comfort', 'inflight_entertainment', 'on-board_service', 'leg_room_service', 'baggage_handling', 'checkin_service', 'inflight_service', 'cleanliness']

df = df.loc[(df[cols_to_filter] != 0).any(axis=1)]



In [ ]:
categorical_columns = ["gender", "customer_type", "type_of_travel", "class"]
# Créer des colonnes hot encoder pour chaque colonne catégorielle
for column in categorical_columns:
    df = pd.get_dummies(df, columns=[column],dtype=int)
df.dropna(inplace=True)
X = df.drop(columns=['satisfaction'])
y = df['satisfaction']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner un modèle CatBoost
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=500, learning_rate=0.1, depth=6, loss_function='Logloss')
model.fit(X_train, y_train)

# Faire des prédictions
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# Précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Précision : {:.2f}%".format(accuracy * 100))

# Matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matrice de confusion :\n", conf_matrix)

# Rapport de classification
class_report = classification_report(y_test, y_pred)
print("Rapport de classification :\n", class_report)

In [ ]:
# Sauvegarde de l'entrainement
model.save_model('modele_catboost.cbm')

from catboost import CatBoostClassifier

# Charger le modèle depuis le fichier
model = CatBoostClassifier()
model.load_model('modele_catboost.cbm')

# transformation des données manuelles
manual_data = {
    'age': 50,
    'flight_distance': 250,
    'inflight_wifi_service': 0,
    'departure_arrival_time_convenient': 0,
    'ease_of_online_booking': 0,
    'gate_location': 0,
    'food_and_drink': 0,
    'online_boarding': 0,
    'seat_comfort': 0,
    'inflight_entertainment': 0,
    'on-board_service': 0,
    'leg_room_service': 0,
    'baggage_handling': 0,
    'checkin_service': 0,
    'inflight_service': 0,
    'cleanliness': 0,
    'arrival_delay_in_minutes': 120,
    'gender_Female': 1,
    'gender_Male': 0,
    'customer_type_Loyal_Customer': 0,
    'customer_type_disloyal_Customer': 1,
    'type_of_travel_Business_travel': 0,
    'type_of_travel_Personal_Travel': 1,
    'class_Business': 0,
    'class_Eco': 1,
    'class_Eco_Plus': 0
}
manual_df = pd.DataFrame([manual_data])

# Faire des prédictions
y_manual_pred = model.predict(manual_df)

In [ ]:
y_manual_pred